In [ ]:
import os
import numpy as np
import pandas as pd
import ast
from shapely.geometry import Point, LineString

# Utility Functions

In [ ]:
def Subset(E,t):
    return E[E.type==t]

def euclidean_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def Dist(ei, ej):
    # Convert single points to a list for uniform handling
    if not isinstance(ei, list):
        ei = [ei]
    if not isinstance(ej, list):
        ej = [ej]
    # print(ei, ej)
    # Calculate minimum distance
    return min(euclidean_distance(a, b) for a in ei for b in ej)


def Closest(e, S):
    min_distance = float('inf')
    closest_point = None

    for _,s in S.iterrows():
        distance = Dist(e.coor, s.coor)
        print(e.coor, s.coor, distance)
        if distance < min_distance:
            min_distance = distance
            closest_point = s

    return closest_point

def ReadFiles(folder_path, symbol='e'):
    elements = pd.DataFrame()

    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            print(file_path)

            # Read the data using pandas
            df = pd.read_csv(file_path, delimiter=';')

            # Convert the 'coor' column to tuples
            df['coor'] = df['coor'].apply(ast.literal_eval)
            df['type'] = filename[2:-5]

            # Append the data to the combined DataFrame
            elements = pd.concat([elements,df], ignore_index=True)
    elements['name'] = [f"{symbol}{i+1}" for i in range(len(elements))]
    elements['next_element'] = None
    elements.insert(0, 'name', elements.pop('name'))

    return elements

In [ ]:
for _,c in Subset(elements, "customer").iterrows():
    e, d = Closest(c, elements[elements.index-Subset(elements, "customers")])
    break
    print(e, d)
    print()

# Raw Information

- I = {
    - i_1: A document containing the DSO's list of elements, their coordinates, and their types. Some elements may be missing, and the coordinates may be inaccurate.

    - i_2: The DSOs lack of information about the customers connections to the network and goal to minimise the cost of connecting customers to the grid. 
    - i_3: A technical meeting stating that for economical and efficiency factors, the connections between two elements must be not too long.
    - i_3: Information regarding the radial configuration of the distribution network.
    - i_4: DSOs goal to reduce energy losses by minimising the path length of each customer.
    - i_5: The knowledge that electricity distributed from one transformer cannot be transmitted to another transformer.  
}

# Well-Defined Information

- $I'$ = {
    - $i'_1 = f_1(i_1)$: Sets of elements, their properties like coordinates and types.
    - $i'_2 = f_2(i_2)$: Elements of type $customer$ are connected to the closest element of type $line$.
    - $i'_3 = f_3(i_3)$: Elements of type $transformer$, $line$ and $switch$ cannot be connected to an element whose distance calculated using the elements GIS coordinates is greater than $R$.
    - $i'_4  = f_4(i_4)$: The total length of a path, defined as the sum of the lengths of the single lines, is at most $L$.  
}

# Read Elements

In [ ]:
#Set of all Real Elements. These elements represent all the elements that is possible to find in the academic example
real_elements = ReadFiles("RealElements")
#Set of all DSO Elements. These elements represent the elements that is DSO knows about
elements = ReadFiles("DSOElements", symbol="hat{e}")

In [ ]:
real_elements

In [ ]:
elements

# Hypothetical Paths

In [ ]:
from itertools import permutations

def calculate_hypothetical_paths(E, C, T):
    R = list(set(E) - set(C) - set(T))
    R.sort()
    print(f"E set: {E}. Len: {len(E)}")
    print(f"C set: {C}. Len: {len(C)}")
    print(f"T set: {T}. Len: {len(T)}")
    print(f"R set: {R}. Len: {len(R)}")
    H = []

    for i in range(len(R) + 1):
        for permutation in permutations(R, i):
            for c in C:
                for t in T:
                    path = [c] + [p for p in permutation] + [t]
                    H.append(path)

    return H

# Example usage with sets E, C, and T
E = elements.index.values
C = Subset(elements,'customer').index.values
# E = ('e_1', 'e_2', 'e_3', 'e_4', 'e_5', 'e_6', 'e_7', 'e_8', 'e_9')
# C = [('e_12')]
T = Subset(elements,'transformer').index.values

hypothetical_paths = calculate_hypothetical_paths(E, C, T)

In [ ]:
print(len(hypothetical_paths))
with open(r'hypotheticalpaths.txt', 'w') as fp:
    # fp.write("\n".join(str(item) for item in hypothetical_paths))
    fp.write("\n".join(str(elements.iloc[item].name.values) for item in hypothetical_paths))

# Hypothetical Paths Compatible with the Well-Defined Information

## $H^{i'_2}$

In [ ]:
for _,c in Subset(elements, "customer").iterrows():
    possible_elements = elements.iloc[elements.index.difference(Subset(elements, "customer").index)]
    e = Closest(c, possible_elements)
    print(c, e)
    print()

In [ ]:
print(c, e)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
fig, ax = plt.subplots(figsize=(10, 8))
df = elements

for index, row in df.iterrows():
    if isinstance(row['coor'], list):
        # Plot lines for coordinate lists
        line_coords = list(zip(*row['coor']))
        ax.plot(line_coords[0], line_coords[1], marker='o', label=row['name'])
    else:
        # Plot single points for tuples
        ax.plot(row['coor'][0], row['coor'][1], marker='o', label=row['name'])

# Add labels and legend
ax.set_xlabel('X-coordinate')
ax.set_ylabel('Y-coordinate')
ax.legend()

# Show the plot
plt.show()

In [ ]:
c = [] 
for h in hypothetical_paths:
    if(h[0]=='e_12' and (h[1]=='e_6')):
        c.append(h)

In [ ]:
print(len(hypothetical_paths))
print(len(c))
print(len(c) / len(hypothetical_paths) * 100)

In [ ]:
#Read files

folder_path = 'Elements'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Filter only CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Initialize an empty DataFrame to store the combined data
elements = pd.DataFrame()

# Read each CSV file and append to the combined DataFrame
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path,header=0, sep=';')
    df['type'] = csv_file[:-5]
    elements = pd.concat([elements,df])
